In [1]:
import pymc3
from pymc3 import Gamma, Model, NUTS, sample, traceplot, Bernoulli
import matplotlib.pyplot as plt
import numpy as np
import gen
import read

%load_ext autoreload
%autoreload 2

In [ ]:
data_path = '../data/'
observation_file = data_path + 'HilNeg 0324 -- Data.csv'
cofactors = read.get_cofactors(data_path + 'cofactors')
path_dict = read.get_model(data_path + 'model2.csv', cofactors = cofactors)
pathways = path_dict.keys()
features = read.get_metabolites(path_dict)
evidence = read.metlin(observation_file)
evidence |= read.hmdb(observation_file)
evidence -= cofactors
features -= cofactors
evidence &= features
reverse_path_dict = read.reverse_dict(path_dict)
metfrag = read.metfrag(observation_file)
metfrag_evidence = read.dict_of_set(read.metfrag_with_scores(observation_file, keep_zero_scores = False), metfrag & features - cofactors - evidence)
evidence = {e : 1 for e in evidence}

features = np.array(list(features))
feat_index = {f : i for i, f in enumerate(features)}
pathways = np.array(list(pathways))
path_index = {p : i for i, p in enumerate(pathways)}

has_evidence = np.array([(f in evidence) for f in features])
rate_prior = 0.5

In [ ]:
m = Model()
import math
print (math.e)
with m:
    #ap = Gamma('ap', [rate_prior] * len(pathways), [1] * len(pathways), shape=86)
    ap  = {p : Gamma(p, rate_prior, 1) for p in pathways}
    bmp = {p : {feat : Gamma('b_{%s, %s}' % (p, feat), ap[p], 1) for feat in path_dict[p]} for i,p in enumerate(pathways)}
    print(len(bmp))
    g = {}
    for feat, f_pathways in reverse_path_dict.items():
        #bm[feat] = sum([bmp[p][feat] for p in f_pathways])
    #g= Bernoulli('g', 1 - math.e ** (-1 * bm[feat]), observed = has_evidence, shape = len(features))
        g[feat] = Bernoulli('g_' + feat, 1 - math.e ** (-1 * sum([bmp[p][feat] for p in f_pathways])), observed = feat in evidence)
        

    trace = sample(10000)

2.718281828459045
86


/opt/conda/lib/python3.5/site-packages/theano/tensor/basic.py:2147: UserWarning: theano.tensor.round() changed its default from `half_away_from_zero` to `half_to_even` to have the same default as NumPy. Use the Theano flag `warn.round=False` to disable this warning.
  "theano.tensor.round() changed its default from"
/opt/conda/lib/python3.5/site-packages/theano/tensor/basic.py:2147: UserWarning: theano.tensor.round() changed its default from `half_away_from_zero` to `half_to_even` to have the same default as NumPy. Use the Theano flag `warn.round=False` to disable this warning.
  "theano.tensor.round() changed its default from"
Auto-assigning NUTS sampler...
Initializing NUTS using advi...


In [ ]:
pm = Model()

with pm:
    ap =  {p.name : Gamma(p.name, p.rate, 1) for p in gen.pathways()}
    bmp = {p.name : {feat : Gamma('b_{%s, %s}' % (p.name, feat), ap[p.name], 1) for feat in p.mets} for p in gen.pathways()}
    for feat, pathways in reverse_path_dict.items():
        bm[feat] = sum([bmp[p][feat] for p in pathways])
    
        

In [ ]:
with pm:
    trace = sample(2000, njobs=4)
pymc3.summary(trace)

In [73]:
with m:
    trace = sample(2000, njobs=4)

Auto-assigning NUTS sampler...
Initializing NUTS using advi...


ValueError: too many values to unpack (expected 1)